# Choose final figures for results
* used to create Figures 2, 3 and 4 from the paper
* add a simple linear regression of LP/FC to risk and benefit scores

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import join
import seaborn as sns
import colorcet as cc

In [ ]:
sns.set(font_scale = 2.5)

In [ ]:
def get_cmap(n, name='tab20b'):
    ### Returns function that maps numbers to colors. 'name' is a colormap name.
    return plt.cm.get_cmap(name, n)

In [ ]:
### define states and their IDs
states = {'state':['Burgenland','Carinthia','Lower Austria','Upper Austria','Salzburg','Styria',
                       'Tyrol','Vorarlberg','Vienna'],'state_id':[1,2,3,4,5,6,7,8,9]}

### english short name versions
doc_list = {'DE':['AM', 'AU', 'CH', 'DER', 'GGH', 'HNO', 'IM', 'KI', 'NEU', 'ORTR','PSY', 'RAD', 'URO'],
            'EN':['GP','OPH','SRG','DER','OBGYN','ENT','IM','PED','NEU','ORTH','PSY','RAD','URO']}

### read risk benefit file and node list file of network
risk_benefit = pd.read_csv('results/Risk_Benefit_table.csv')
NL = pd.read_csv('results/PSNW_NodeList.csv',encoding='ISO-8859-15')
final = NL.join(risk_benefit.set_index('docid'),on='docid').drop(columns='specialty')

### risk and benefit
final_gr = final.groupby('fg').agg({'Risk':['mean','std'],'Benefit':['mean','std']}) * (-1)
final_gr.loc[:,('Risk','std')] = final_gr.Risk['std'].values*(-1)
final_gr.loc[:,('Benefit','std')] = final_gr.Benefit['std'].values*(-1)

final_gr = final.groupby(['state_ID','fg']).agg({'Risk':['mean','std'],'Benefit':['mean','std']}).reset_index()

In [ ]:
### read results dataframe
df = pd.read_csv('results/DF_results_Final.csv')

### select df up to limit
LPlimit = 1
FClimit = 20

LP = df[df.lost_patients_state<=LPlimit][['run','state','specialty','shock','lost_patients_state','perc_docs_removed']]
FC = df[df.remaining_FC_filled<=FClimit][['run','state','specialty','shock','remaining_FC_filled','perc_docs_removed']]

### lost patients per state and specialist for selected limit
idx = LP.groupby(['state','specialty','run'])['lost_patients_state'].transform(max) == LP['lost_patients_state']
LP = LP[idx]
LP = LP.sort_values(['state','specialty','run','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty','run'], keep='last')[['run','state','specialty','shock','lost_patients_state',
                                             'perc_docs_removed']]

LP.reset_index(drop=True,inplace=True)

### lost patients per state and specialist for selected limit
idx = FC.groupby(['state','specialty','run'])['remaining_FC_filled'].transform(max) == FC['remaining_FC_filled']
FC = FC[idx]
FC = FC.sort_values(['state','specialty','run','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty','run'], keep='last')[['run','state','specialty','shock','remaining_FC_filled',
                                             'perc_docs_removed']]

FC.reset_index(drop=True,inplace=True)

### mean over all runs
LP = LP.groupby(['state','specialty']).agg({'perc_docs_removed':'mean'}).reset_index()
FC = FC.groupby(['state','specialty']).agg({'perc_docs_removed':'mean'}).reset_index()

### mean and std over all states
FC = FC.groupby('specialty').agg({'perc_docs_removed':['mean','std']})
LP = LP.groupby('specialty').agg({'perc_docs_removed':['mean','std']})

## Scatter plots (Figure 4)
* LP vs. FC limits for %docs removed

In [ ]:
### risk benefit
final_part = final.groupby(['fg']).agg({'Risk':['mean','std'],'Benefit':['mean','std']}).reset_index()
cmap = get_cmap(len(final_part[('Benefit','mean')].values))

In [ ]:
font = 28
fig, ax = plt.subplots(1,2,figsize=(19,8))
for i in range(0,len(final_part[('Benefit','mean')].values)):
    ax[1].errorbar(x=final_part[('Risk','mean')].values[i],y = final_part[('Benefit','mean')].values[i],fmt='o',
                 xerr=final_part[('Risk','std')].values[i],yerr=final_part[('Benefit','std')].values[i],
                 capsize=6, ms=10, capthick=3,elinewidth=2,c=cmap(i))
    
for i in range(0,len(FC[('perc_docs_removed','mean')].values)):
    ax[0].errorbar(x=FC[('perc_docs_removed','mean')].values[i],y = LP[('perc_docs_removed','mean')].values[i],fmt='o',
                 xerr=FC[('perc_docs_removed','std')].values[i],yerr=LP[('perc_docs_removed','std')].values[i],
                 capsize=6, ms=10, capthick=3,elinewidth=2,c=cmap(i))


### add a) b) to panels
ax[0].annotate('a)', xy=(0.03,0.92),xycoords='figure fraction', fontsize=26)
ax[1].annotate('b)', xy=(0.46,0.92),xycoords='figure fraction', fontsize=26)
    
#plt.suptitle('Austria',fontsize=font)
ax[0].set_xlabel(r'$L_{FC}$ [%]',fontsize=font)
ax[0].set_ylabel(r'$L_{LP}$ [%]',fontsize=font)
ax[0].set_xlim([0,100])   #baseline setting [8,20]
ax[0].set_ylim([0,100])   #baseline setting [5,60]


ax[1].set_xlabel('Risk',fontsize=font)
ax[1].set_ylabel('Benefit',fontsize=font)
ax[1].set_xlim([0,1])      #baseline setting [0.3,0.9]
ax[1].set_ylim([0,1])      #baseline setting [0,1]
plt.legend(doc_list['EN'],bbox_to_anchor=(1.4, 1),fontsize=20)
plt.tight_layout()
fig.subplots_adjust(wspace=0.25)

plt.savefig('figures/Final_Scatter.svg',format='svg')
plt.savefig('figures/Final_Scatter.png',format='png',dpi=300)

plt.show()

## Line plots (Figure 2 and SI Figures)

In [ ]:
# information selection criteria
patient_type = 'total'
capacity_type = 'hour-based'
timeframe = 'quarterly'

### read doc info and sim info
N = pd.read_excel('results/states_doc_info_{}_{}_{}.xlsx'.format(patient_type, capacity_type, timeframe))
dta = pd.read_csv('results/DF_results_Final.csv')
N.set_index('state',inplace=True)

### rename states to english
dta.loc[dta.state=='Burgenland','state'] = 'Burgenland'
dta.loc[dta.state=='Kärnten','state'] = 'Carinthia'
dta.loc[dta.state=='Niederösterreich','state'] = 'Lower Austria'
dta.loc[dta.state=='Oberösterreich','state'] = 'Upper Austria'
dta.loc[dta.state=='Salzburg','state'] = 'Salzburg'
dta.loc[dta.state=='Steiermark','state'] = 'Styria'
dta.loc[dta.state=='Tirol','state'] = 'Tyrol'
dta.loc[dta.state=='Vorarlberg','state'] = 'Vorarlberg'
dta.loc[dta.state=='Wien','state'] = 'Vienna'

In [ ]:
### combined lineplots
font = 26
sns.set(font_scale = 2)

for doc in dta.specialty.unique():
    fig, ax = plt.subplots(1,2,figsize=(14,6.5))
    ### lost patients (summed up over shocks!)
    sns.lineplot(ax = ax[0], x = 'perc_docs_removed',y = 'lost_patients_state',data=dta[dta.specialty==doc],
                 hue='state')
    ax[0].set_ylabel('Lost patients [%]',fontsize=font)
    ax[0].set_xlabel('Removed physicians [%]',fontsize=font)
    ax[0].plot([0,100],[LPlimit,LPlimit],'k--')


    ### remaining free capacity
    sns.lineplot(ax = ax[1], data = dta[dta.specialty==doc], x = 'perc_docs_removed',y = 'remaining_FC_filled',
                 hue = 'state',legend=False)
    ax[1].set_ylabel('Free capacity filled [%]',fontsize=font)
    ax[1].set_xlabel('Removed physicians [%]',fontsize=font)
    ax[1].plot([0,100],[FClimit,FClimit],'k--')

    ### add a) b) to panels
    ax[0].annotate('a)', xy=(0.03,0.86),xycoords='figure fraction', fontsize=22)
    ax[1].annotate('b)', xy=(0.51,0.86),xycoords='figure fraction', fontsize=22)

    plt.suptitle(doc_list['EN'][doc_list['DE'].index(doc)],fontsize=font)
    ax[0].legend(loc='upper left',fontsize=font-10)
    plt.tight_layout(h_pad=2)
    plt.subplots_adjust(top=0.85)

    plt.savefig('figures/LPFC_states_main_{}.svg'.format(doc),bbox_inches = 'tight',format = 'svg')
    plt.savefig('figures/LPFC_states_main_{}.PNG'.format(doc),bbox_inches = 'tight',format = 'png',dpi=200)
    plt.show()

# Heatmaps (Figure 3)
* critical limits for LP and FC for state vs. speciality 

In [ ]:
### read results dataframe 
df = pd.read_csv('results/DF_results_Final.csv')

### select df up to limit
LPlimit = 1
FClimit = 20

LP = df[df.lost_patients_state<=LPlimit][['run','state','specialty','shock','lost_patients_state','perc_docs_removed']]
FC = df[df.remaining_FC_filled<=FClimit][['run','state','specialty','shock','remaining_FC_filled','perc_docs_removed']]

### rename to english states
LP.loc[LP.state=='Kärnten','state'] = 'Carinthia'
LP.loc[LP.state=='Niederösterreich','state'] = 'Lower Austria'
LP.loc[LP.state=='Oberösterreich','state'] = 'Upper Austria'
LP.loc[LP.state=='Steiermark','state'] = 'Styria'
LP.loc[LP.state=='Tirol','state'] = 'Tyrol'
LP.loc[LP.state=='Wien','state'] = 'Vienna'

FC.loc[FC.state=='Kärnten','state'] = 'Carinthia'
FC.loc[FC.state=='Niederösterreich','state'] = 'Lower Austria'
FC.loc[FC.state=='Oberösterreich','state'] = 'Upper Austria'
FC.loc[FC.state=='Steiermark','state'] = 'Styria'
FC.loc[FC.state=='Tirol','state'] = 'Tyrol'
FC.loc[FC.state=='Wien','state'] = 'Vienna'

In [ ]:
### lost patients per state and specialist for selected limit
idx = LP.groupby(['state','specialty','run'])['lost_patients_state'].transform(max) == LP['lost_patients_state']
LP = LP[idx]
LP = LP.sort_values(['state','specialty','run','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty','run'], keep='last')[['run','state','specialty','shock','lost_patients_state',
                                             'perc_docs_removed']]

LP.reset_index(drop=True,inplace=True)

### lost patients per state and specialist for selected limit
idx = FC.groupby(['state','specialty','run'])['remaining_FC_filled'].transform(max) == FC['remaining_FC_filled']
FC = FC[idx]
FC = FC.sort_values(['state','specialty','run','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty','run'], keep='last')[['run','state','specialty','shock','remaining_FC_filled',
                                             'perc_docs_removed']]

FC.reset_index(drop=True,inplace=True)

In [ ]:
### mean and stand.deviation nof perc_docs_removed over all runs
LP = LP.groupby(['state','specialty']).agg({'perc_docs_removed':['mean','std']}).reset_index()
FC = FC.groupby(['state','specialty']).agg({'perc_docs_removed':['mean','std']}).reset_index()

### drop multicolumns names
LP.columns = ['state','specialty','perc_docs_removed_mean','perc_docs_removed_std']
FC.columns = ['state','specialty','perc_docs_removed_mean','perc_docs_removed_std']

### add info on state ID
states = {'state':['Burgenland','Carinthia','Lower Austria','Upper Austria',
                       'Salzburg','Styria','Tyrol','Vorarlberg','Vienna'],'state_id':[1,2,3,4,5,6,7,8,9]}

LP['state_ID'] = 0
FC['state_ID'] = 0
for s,i in zip(states['state'],states['state_id']):
    LP.loc[LP.state==s,'state_ID'] = i
    FC.loc[FC.state==s,'state_ID'] = i

### switch order
LP = LP[['state', 'state_ID', 'specialty','perc_docs_removed_mean','perc_docs_removed_std']]
FC = FC[['state', 'state_ID', 'specialty','perc_docs_removed_mean','perc_docs_removed_std']]

### dataframes for heatmaps
LPheat = pd.DataFrame(columns = LP.state.unique(),index = doc_list['EN'])
FCheat = pd.DataFrame(columns = LP.state.unique(),index = doc_list['EN'])

### fill in
for fg in doc_list['EN']:
    for st in LP.state.unique():
        if len(LP.loc[(LP.state==st)&(LP.specialty==doc_list['DE'][doc_list['EN'].index(fg)]),'perc_docs_removed_mean'])>0:
            LPheat.loc[fg,st] = float(LP.loc[(LP.state==st)&(LP.specialty==doc_list['DE'][doc_list['EN'].index(fg)]),'perc_docs_removed_mean'].item())
            LPheat[st] = LPheat[st].astype(float)
        if len(FC.loc[(FC.state==st)&(FC.specialty==doc_list['DE'][doc_list['EN'].index(fg)]),'perc_docs_removed_mean'])>0:
            FCheat.loc[fg,st] = float(FC.loc[(FC.state==st)&(FC.specialty==doc_list['DE'][doc_list['EN'].index(fg)]),'perc_docs_removed_mean'].item())
            FCheat[st] = FCheat[st].astype(float)

FCheat = FCheat.sort_values(by='Burgenland')
LPheat = LPheat.reindex(FCheat.index.values)

In [ ]:
import seaborn as sns
sns.set(font_scale=1)
f, (ax1,ax2) = plt.subplots(1,2,figsize=(16,6))

### heatmap for free capacity
h1 = sns.heatmap(FCheat,annot=True,linewidths=0.5,vmin=0,vmax=np.ceil(FCheat.max().max()),cmap='cividis',ax=ax1,cbar_kws={'label':'Removed physicians [%]'})
ax1.xaxis.tick_top()
h1.set_yticklabels(h1.get_yticklabels(),rotation=0,fontsize=14)
h1.set_xticklabels(h1.get_xticklabels(),rotation=45,fontsize=14,horizontalalignment='left')
h1.set_title(r'$L_{FC}$',fontsize=18, y=-0.1)
h1.figure.axes[-1].yaxis.label.set_size(18)


### heatmap for lost patients
h2 = sns.heatmap(LPheat,annot=True,linewidths=0.5,vmin=0,vmax=np.ceil(LPheat.max().max()),cmap='viridis',ax=ax2,
                 cbar_kws={'label':'Removed physicians [%]'})
ax2.xaxis.tick_top()
h2.set_yticklabels(h2.get_yticklabels(),rotation=0,fontsize=14)
h2.set_xticklabels(h2.get_xticklabels(),rotation=45,fontsize=14,horizontalalignment='left')
h2.set_title(r'$L_{LP}$',fontsize=18, y=-0.1)
h2.figure.axes[-1].yaxis.label.set_size(18)

### add a) b) to panels
ax1.annotate('a)', xy=(0.045,0.92),xycoords='figure fraction', fontsize=22)
ax2.annotate('b)', xy=(0.53,0.92),xycoords='figure fraction', fontsize=22)

plt.savefig('figures/LPFC_heatmap.svg',bbox_inches = 'tight',format = 'svg')
plt.savefig('figures/LPFC_heatmap.PNG',bbox_inches = 'tight',format = 'png',dpi=200)
plt.show()

## Linear regression for LP/FC to risk and benefit
* LP/FC ~ < risk > + < benefit > + const.
* for every state & specialty
* only gives meaningful results with entire dataset, otherwise NaN for missing values in states and specialties

In [ ]:
### combine all info into one dataframe
FC['RiskScore'] = 0
FC['BenefitScore'] = 0
FC['LP'] = 0
FC['FC'] = 0

for st in FC.state_ID.unique():
    for d in FC.specialty.unique():
        # risk & benefit
        if len(final_gr.loc[(final_gr.state_ID==st)&(final_gr.fg==d),('Risk','mean')])>0:
            FC.loc[(FC.state_ID==st)&(FC.specialty==d),'RiskScore'] = final_gr.loc[(final_gr.state_ID==st)&
                                                                        (final_gr.fg==d),('Risk','mean')].item()
        if len(final_gr.loc[(final_gr.state_ID==st)&(final_gr.fg==d),('Benefit','mean')])>0:
            FC.loc[(FC.state_ID==st)&(FC.specialty==d),'BenefitScore'] = final_gr.loc[(final_gr.state_ID==st)&
                                                                        (final_gr.fg==d),('Benefit','mean')].item()
        
        # LP & FC
        if len(FC.loc[(FC.state_ID==st)&(FC.specialty==d),'perc_docs_removed_mean'])>0:
            FC.loc[(FC.state_ID==st)&(FC.specialty==d),'FC'] = FC.loc[(FC.state_ID==st)&(FC.specialty==d),
                                                                  'perc_docs_removed_mean'].item()
        if len(LP.loc[(LP.state_ID==st)&(LP.specialty==d),'perc_docs_removed_mean'])>0:
            FC.loc[(FC.state_ID==st)&(FC.specialty==d),'LP'] = LP.loc[(LP.state_ID==st)&(LP.specialty==d),
                                                                  'perc_docs_removed_mean'].item()

In [ ]:
#FC.drop(columns=['num_docs_removed','perc_free_capacity','perc_docs_removed'],inplace=True)
df = FC.copy()

In [ ]:
### Linear regression
from sklearn.linear_model import LinearRegression
from IPython.display import display

reg = pd.DataFrame(columns=['LP','FC'],index=['RiskScore','BenefitScore'])

for out in ['FC','LP']:
    X = df[['RiskScore','BenefitScore']].copy()
    y = df[out].copy()

    if np.all(X.isna())==True:
        res = LinearRegression(normalize=True).fit(X,y)

        reg.loc['RiskScore',out] = res.coef_[0]
        reg.loc['BenefitScore',out] = res.coef_[1]
display(reg)

In [ ]:
### speciality specific
reg_all = pd.DataFrame()
    
for sp in df.specialty.unique():
    reg = pd.DataFrame(columns=['LP','FC','specialty'],index=['RiskScore','BenefitScore'])
    for out in ['FC','LP']:
        X = df.loc[df.specialty==sp,['RiskScore','BenefitScore']].copy()
        y = df.loc[df.specialty==sp,out].copy()

        if np.all(X.isna())==True:
            res = LinearRegression(normalize=True).fit(X,y)

            reg.loc['RiskScore',out] = res.coef_[0]
            reg.loc['BenefitScore',out] = res.coef_[1]
            reg.specialty = sp
    reg_all = pd.concat([reg_all,reg])
    print('---------  '+sp+'  -----------')
    display(reg)

In [ ]:
reg_all = reg_all.reset_index()
reg_all.rename(columns={'index':'score'},inplace=True)
reg_all.LP = reg_all.LP.astype(float)
reg_all.FC = reg_all.FC.astype(float)

In [ ]:
### averaged coefficient sizes for risk and benefit scores over all specialties
display(np.round(reg_all[['LP','FC','score']].groupby('score').mean(),0))
display(np.round(reg_all[['LP','FC','score']].groupby('score').std(),0))